In [17]:
import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
df=pd.read_csv(r"C:\Users\ADITYA SINGH\Downloads\NLP_train.csv")
print(df.head())

                 id                                       comment_text  toxic
0  e617e2489abe9bca  "\r\n\r\n A barnstar for you! \r\n\r\n  The De...      0
1  9250cf637294e09d  "\r\n\r\nThis seems unbalanced.  whatever I ha...      0
2  ce1aa4592d5240ca  Marya Dzmitruk was born in Minsk, Belarus in M...      0
3  48105766ff7f075b      "\r\n\r\nTalkback\r\n\r\n Dear Celestia...  "      0
4  0543d4f82e5470b6  New Categories \r\n\r\nI honestly think that w...      0


In [19]:
df.drop(columns = ['id'], inplace=True)
df.head()

,comment_text,toxic
0,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,New Categories \r\n\r\nI honestly think that w...,0


In [20]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
ps = PorterStemmer()
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words:
            token=ps.stem(token)
            result.append(token)
            
    return result
df["clean"]=df.comment_text.apply(preprocess)
df.clean.head()

0    [barnstar, defend, wiki, barnstar, like, edit,...
1    [seem, unbalanc, whatev, said, mathsci, said, ...
2    [marya, dzmitruk, born, minsk, belaru, march, ...
3                           [talkback, dear, celestia]
4    [new, categori, honestli, think, need, add, ca...
Name: clean, dtype: object

In [22]:
df.clean[0]

['barnstar',
 'defend',
 'wiki',
 'barnstar',
 'like',
 'edit',
 'kayastha',
 'page',
 'let',
 'form',
 'solidar',
 'group',
 'malign',
 'articl',
 'subject',
 'matter',
 'propos',
 'follo',
 'name',
 'group',
 'unit',
 'intellectu',
 'front',
 'kayastha',
 'ethinicti',
 'racist',
 'castist',
 'abus',
 'uifkearca']

In [23]:
df["joined"]=[" ".join(token) for token in df['clean'].values]
l=[]
for i in df.joined:
    l.append(i)
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, analyzer='word')    
x=np.array(tfidf.fit_transform(l).toarray())

In [24]:
x.shape

(5000, 212)

In [25]:
df.clean.head()

0    [barnstar, defend, wiki, barnstar, like, edit,...
1    [seem, unbalanc, whatev, said, mathsci, said, ...
2    [marya, dzmitruk, born, minsk, belaru, march, ...
3                           [talkback, dear, celestia]
4    [new, categori, honestli, think, need, add, ca...
Name: clean, dtype: object

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, df.toxic, test_size = 0.3)

In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train1 = sc.fit_transform(x_train)
x_test1 = sc.transform(x_test)

In [28]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(x_train1, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [29]:
y_pred = classifier.predict(x_test1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1365,    0],
       [ 135,    0]], dtype=int64)

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
clfr = GridSearchCV(SVC(kernel='linear', gamma='auto',class_weight={0: 0.7, 1:6}), {
    'C': [1,10,20,30,40,50,60]
}, cv=5, return_train_score=False)
clfr.fit(x_train1, y_train)
data= pd.DataFrame(clfr.cv_results_)
print(data[['param_C','mean_test_score']])
print(clfr.best_score_)

  param_C  mean_test_score
0       1         0.818000
1      10         0.817429
2      20         0.817143
3      30         0.817714
4      40         0.817714
5      50         0.818000
6      60         0.818000
0.818


In [43]:
clas=SVC(kernel='linear', class_weight={0: 0.7, 1 : 6}, gamma="auto", C=1.0 )
clas.fit(x_train1, y_train)
y_pred=clas.predict(x_test1)
cm1=confusion_matrix(y_test, y_pred)
cm1

array([[1080,  285],
       [  59,   76]], dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.7706666666666667 0.3064516129032258 0.562962962962963
